In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [6]:
model_id = "microsoft/phi-1_5"
model_revision = "349cf8b5e81fd5f791d1740da5de1313a0419bbd" # latest as of feb 1st

In [4]:
tokenizer = AutoTokenizer.from_pretrained(base_model_id, trust_remote_code=True) # use_fast=True?
# tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json: 100%|████████████████████████████████████████████████████████████████████████████████████████| 237/237 [00:00<00:00, 232kB/s]
vocab.json: 100%|████████████████████████████████████████████████████████████████████████████████████████████████| 798k/798k [00:00<00:00, 3.10MB/s]
merges.txt: 100%|████████████████████████████████████████████████████████████████████████████████████████████████| 456k/456k [00:00<00:00, 5.89MB/s]
tokenizer.json: 100%|██████████████████████████████████████████████████████████████████████████████████████████| 2.11M/2.11M [00:00<00:00, 9.90MB/s]
added_tokens.json: 100%|███████████████████████████████████████████████████████████████████████████████████████| 1.08k/1.08k [00:00<00:00, 1.68MB/s]
special_tokens_map.json: 100%|████████████████████████████████████████████████████████████████████████████████████| 99.0/99.0 [00:00<00:00, 237kB/s]


In [12]:
vocab = tokenizer.get_vocab()

In [13]:
len(vocab)

50295

In [26]:
vocab["def"]

4299

In [7]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    revision=model_revision,
    trust_remote_code=True,
    # torch_dtype=compute_dtype,
    # quantization_config=bnb_config,
)

config.json: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 864/864 [00:00<00:00, 1.40MB/s]
configuration_phi.py: 100%|████████████████████████████████████████████████████████████████████████████████████| 9.26k/9.26k [00:00<00:00, 16.1MB/s]
modeling_phi.py: 100%|█████████████████████████████████████████████████████████████████████████████████████████| 62.7k/62.7k [00:00<00:00, 38.3MB/s]
pytorch_model.bin: 100%|███████████████████████████████████████████████████████████████████████████████████████| 2.84G/2.84G [03:21<00:00, 14.0MB/s]
generation_config.json: 100%|█████████████████████████████████████████████████████████████████████████████████████| 74.0/74.0 [00:00<00:00, 265kB/s]


In [ ]:
model = AutoModelForCausalLM.from_pretrained("microsoft/phi-1_5", torch_dtype="auto", trust_remote_code=True)

In [ ]:
model = AutoModelForCausalLM.from_pretrained("microsoft/phi-1_5", torch_dtype="auto", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-1_5", trust_remote_code=True)